# integrate.ai API LSTM Sample Notebook

## Prerequisites:
An instance of our docker client, downloaded from the Docker page in the UI  
An IAI token, created using the Token Management page in the UI

## Set environment variables (or replace inline) with your IAI Token

In [ ]:
import os

IAI_TOKEN = os.environ.get("IAI_TOKEN")

## Authenticate to the integrate.ai api client

In [ ]:
from integrate_ai_sdk.api import connect

client = connect(token=IAI_TOKEN)

## Custom model, dataset, and LSTMTagger.json
Choose a name for your custom model, and set the path for the model and data configurations.  
Note that the name for your custom model **must be unique**.  
This means that the name for your custom model cannot already be in the Package Name column of the Custom Models Packages Table in the Model Library Page of the UI.


In [ ]:
# Update the following lines to your package name and model and data configuration paths
package_name = "lstm_sample_package"
model_config_path = "../lstmTagger/lstmtagger.json"
data_config_path = "../lstmTagger/taggerDataset.json"

# Update the following lines to your package and dataset paths
package_path = "../lstmTagger"
dataset_path = "../lstmTagger/sample_data"

In [ ]:
import json

with open(model_config_path, "r") as f:
    lstm_model_config = json.load(f)

with open(data_config_path, "r") as f:
    data_schema = json.load(f)

## Upload customized model


In [ ]:
client.upload_model(
    package_path=package_path,
    dataset_path=dataset_path,
    package_name=package_name,
    sample_model_config_path=model_config_path,
    sample_data_config_path=data_config_path,
    batch_size=256,
    task="classification",
    test_only=False,
    description="A custom LSTM model.",
)

## Local Taskrunner

In [ ]:
from integrate_ai_sdk.taskgroup.base import SessionTaskGroup
from integrate_ai_sdk.taskgroup.taskbuilder.local import local_python

local_tb = local_python(client=client)

## Create a Session

The Quickstart guide for [creating a session](https://integrate-ai.gitbook.io/integrate.ai-user-documentation/tutorials/end-user-tutorials/model-training-with-a-sample-local-dataset#create-and-start-the-session) gives a bit more context into the paramters that are used during session creation.<br />
For this session we are going to be using two training clients and two rounds. 

In [ ]:
model_config = {
    "strategy": {"name": "FedAvg", "params": {}},
    "model": {"params": lstm_model_config},
    "ml_task": {"type": "classification", "params": {}},
    "optimizer": {"name": "SGD", "params": {"learning_rate": 0.9, "momentum": 0.9}},
    "differential_privacy_params": {"epsilon": 4, "max_grad_norm": 7},
    "seed": 23,  # for reproducibility
}

In [ ]:
session = client.create_fl_session(
    name="LSTM custom model notebook",
    description="Training a custom LSTM model using a jupyter notebook.",
    min_num_clients=2,
    num_rounds=5,
    package_name=package_name,
    model_config=model_config,
    data_config=data_schema,
).start()

session.id

## Start a training session using iai local taskbuilder
This example uses a local taskbuilder, so that you can use files in your local environment. If you would like to use data in the cloud use an AWS or Azure taskrunner (as shown in other example notebooks).

In [ ]:
from os.path import abspath

# The path to the data you want to train should be an absolute path to the directory
data_path = abspath(dataset_path)

# client_1 = subprocess.Popen(
#     f"iai client train --token {IAI_TOKEN} --session {session.id} --train-path {data_path} --test-path {data_path} --batch-size 512 --approve-custom-package --client-name client-1 --remove-after-complete",
#     shell=True,
#     stdout=subprocess.PIPE,
#     stderr=subprocess.PIPE,
# )
# client_2 = subprocess.Popen(
#     f"iai client train --token {IAI_TOKEN} --session {session.id} --train-path {data_path} --test-path {data_path} --batch-size 512 --approve-custom-package --client-name client-2 --remove-after-complete",
#     shell=True,
#     stdout=subprocess.PIPE,
#     stderr=subprocess.PIPE,
# )

session_task_group_context = (
        SessionTaskGroup(session) \
        .add_task(local_tb.hfl(train_path=data_path, test_path=data_path))\
        .add_task(local_tb.hfl(train_path=data_path, test_path=data_path))\
        .start()
    )


## Poll for session status

In [ ]:
# Check the task group status

for i in session_task_group_context.contexts.values():
    print(json.dumps(i.status(), indent=4))

session_task_group_context.monitor_task_logs()

In [ ]:
# Wait for the tasks to complete (success = True)

session_task_group_context.wait(60*8, 2)

## Session Complete!

You can view the results using the following code.

In [ ]:
fig = session.metrics().plot()

You can view the training metrics via the .metrics() method.

In [ ]:
display(session.metrics().as_dict())  # view all of the training metrics
display(session.metrics().federated_metrics)  # view the loss for each round

## Trained model weights are accessible from the completed session
Model parameters can be retrieved using the model's state_dict method. These parameters can then be saved using torch.save().

In [ ]:
import torch

model = session.model().as_pytorch()

save_state_dict_folder = "./saved_models"
# PyTorch conventional file type
file_name = f"{session.id}.pt"
os.makedirs(save_state_dict_folder, exist_ok=True)
saved_state_dict_path = os.path.join(save_state_dict_folder, file_name)

with open(saved_state_dict_path, "w") as f:
    torch.save(model.state_dict(), saved_state_dict_path)

## Load the saved model

To load a model saved previously, a model object needs to be initialized first. This can be done by directly importing one of the IAI-supported packages (e.g., FFNet) or using the model class defined in a custom package. 

In [ ]:
from integrate_ai_sdk.sample_packages.lstmTagger.model import LSTMTagger

model = LSTMTagger(embedding_dim=4, hidden_dim=3, output_size=4, vocab_size=9)

# use torch.load to unpickle the state_dict
target_state_dict = torch.load(saved_state_dict_path)

model.load_state_dict(target_state_dict)

## Load the test data



In [ ]:
from integrate_ai_sdk.sample_packages.lstmTagger.dataset import TaggerDataset
from torch.utils.data import DataLoader

ds = TaggerDataset(path=dataset_path, max_len=5)
dl = DataLoader(ds)

## Run predictions using the federated model

In [ ]:
model.eval()
y_true = torch.tensor([])
y_pred = torch.tensor([])
for x, y in dl:
    y_true = torch.cat((y_true, y))

    # The following line calculates the predicted label for a classification problem, and should be modified to fit your needs
    pred = model(x).max(dim=1)[1]

    y_pred = torch.cat((y_pred, pred))

In [ ]:
f"accuracy: {(y_pred == y_true).float().mean().item()}"